In [1]:
import torch
import numpy as np
import random
from sklearn.metrics import classification_report
class XORNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = torch.nn.Linear(2,2)
        self.linear2 = torch.nn.Linear(2,2)
    def forward(self, x):
        output = self.linear1(x)
        output = torch.nn.functional.relu(output)
        output = self.linear2(output)
        return output

def gen_XOR_dataset(n:int = 100):
    """
        we agree that signal that is less than 0.3 is False, greater than 0.7 is True
    """
    inputs = np.random.uniform(0, 1, n)

    dataset = []
    for i in inputs:
        for j in inputs:
            if i <= 0.3 and j<=0.3: #00
                dataset.append([[i,j], 0])
            elif i<=0.3 and j>=0.7: #01
                dataset.append([[i,j], 1])
            elif i>=0.7 and j<=0.3: #10
                dataset.append([[i,j], 1])
            elif i>=0.7 and j>=0.7: #11
                dataset.append([[i,j], 0])

    random.shuffle(dataset)    
    return dataset

dataset = gen_XOR_dataset()
print(len(dataset))

4225


In [11]:
xornet  = XORNet()
crit  = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(xornet.parameters())

n_train = int(len(dataset)*0.7)
trainset = dataset[:n_train]
testset = dataset[n_train:]
traintensor = torch.stack([torch.Tensor(input) for input, label in trainset])
trainlabels = torch.LongTensor([output for input, output in trainset])
print(traintensor.shape)
print(trainlabels.shape)


torch.Size([2957, 2])
torch.Size([2957])


In [12]:


EPOCH = 10000
for e in range(EPOCH):
    optimizer.zero_grad()
    pred = xornet(traintensor)
    loss = crit(pred, trainlabels)
    if e%1000 == 0:
        print(loss)
        print("sample", traintensor[0])
        print("sample pred", pred[0])

    loss.backward()
    optimizer.step()


tensor(0.8650, grad_fn=<NllLossBackward>)
sample tensor([0.0312, 0.1243])
sample pred tensor([ 0.3533, -0.8217], grad_fn=<SelectBackward>)
tensor(0.6072, grad_fn=<NllLossBackward>)
sample tensor([0.0312, 0.1243])
sample pred tensor([ 0.7136, -0.9146], grad_fn=<SelectBackward>)
tensor(0.2291, grad_fn=<NllLossBackward>)
sample tensor([0.0312, 0.1243])
sample pred tensor([ 1.7780, -1.9224], grad_fn=<SelectBackward>)
tensor(0.0787, grad_fn=<NllLossBackward>)
sample tensor([0.0312, 0.1243])
sample pred tensor([ 2.7800, -2.9040], grad_fn=<SelectBackward>)
tensor(0.0369, grad_fn=<NllLossBackward>)
sample tensor([0.0312, 0.1243])
sample pred tensor([ 3.7165, -3.8265], grad_fn=<SelectBackward>)
tensor(0.0200, grad_fn=<NllLossBackward>)
sample tensor([0.0312, 0.1243])
sample pred tensor([ 4.6101, -4.7089], grad_fn=<SelectBackward>)
tensor(0.0116, grad_fn=<NllLossBackward>)
sample tensor([0.0312, 0.1243])
sample pred tensor([ 5.5078, -5.5969], grad_fn=<SelectBackward>)
tensor(0.0070, grad_fn=<Nll

In [15]:
def evaluate(testset, xornet):
    testtensor = torch.stack([torch.Tensor(input) for input, label in testset])
    testlabels = torch.LongTensor([output for input, output in testset])
    pred = xornet(testtensor)
    pred = torch.argmax(pred, dim=1)
    print(classification_report(testlabels, pred))

evaluate(testset, xornet)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       683
           1       1.00      1.00      1.00       585

    accuracy                           1.00      1268
   macro avg       1.00      1.00      1.00      1268
weighted avg       1.00      1.00      1.00      1268

